## Linear Regression with PyTorch


![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

In [1]:
pip install jovian --upgrade

     |████████████████████████████████| 94 kB 1.6 MB/s eta 0:00:011
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6500 sha256=66860d6dfcb5a7f7b490f7b26d6ae38586a1c4d3ec9e0319151a8c0d226c7827
  Stored in directory: /root/.cache/pip/wheels/2a/ea/87/dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid
Note: you may need to restart the kernel to use updated packages.


We begin by importing Numpy and PyTorch:

In [2]:
# Uncomment the command below if Numpy or PyTorch is not installed
# !conda install numpy pytorch cpuonly -c pytorch -y

In [3]:
import numpy as np
import torch

## Training data

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:

In [6]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [7]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.3983, -1.3742,  1.1659],
        [-0.1273, -1.3044, -2.2683]], requires_grad=True)
tensor([-2.3778, -1.0912], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [8]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [9]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -73.3924, -195.3162],
        [ -84.9363, -272.6340],
        [-153.5511, -318.5183],
        [ -58.9588, -154.0929],
        [ -80.1712, -293.8778]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [10]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [11]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor, and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [12]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(77951.5781, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834)*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the *loss*, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [13]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [14]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.3983, -1.3742,  1.1659],
        [-0.1273, -1.3044, -2.2683]], requires_grad=True)
tensor([[-13830.8828, -16171.8418,  -9561.8457],
        [-28126.1836, -31802.5410, -19496.2246]])


The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [15]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [16]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -73.3924, -195.3162],
        [ -84.9363, -272.6340],
        [-153.5511, -318.5183],
        [ -58.9588, -154.0929],
        [ -80.1712, -293.8778]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [17]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(77951.5781, grad_fn=<DivBackward0>)


In [18]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-13830.8828, -16171.8418,  -9561.8457],
        [-28126.1836, -31802.5410, -19496.2246]])
tensor([-166.4020, -338.8879])


Finally, we update the weights and biases using the gradients computed above.

In [19]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

* We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

* We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the *learning rate* of the algorithm. 

* After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [20]:
print(w)
print(b)

tensor([[-0.2600, -1.2125,  1.2615],
        [ 0.1539, -0.9864, -2.0733]], requires_grad=True)
tensor([-2.3761, -1.0878], requires_grad=True)


With the new weights and biases, the model should have lower loss.

In [21]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(53121.9375, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [22]:
# Train for 100 epochs
for i in range(2000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [23]:
w

tensor([[-0.4228,  0.7803,  0.8739],
        [-0.2209,  0.9145,  0.6123]], requires_grad=True)

In [24]:
b

tensor([-2.3801, -1.0736], requires_grad=True)

Once again, let's verify that the loss is now lower:

In [25]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(7.4157, grad_fn=<DivBackward0>)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [26]:
# Predictions
preds

tensor([[ 56.6136,  70.4025],
        [ 83.7411,  98.4891],
        [116.0831, 137.7653],
        [ 20.3827,  38.3765],
        [104.5276, 114.3374]], grad_fn=<AddBackward0>)

In [27]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs. 

At this point, we can save our notebook and upload it to [Jovian.ml](https://www.jovian.ml) for future reference and sharing.

In [28]:
# Train for 100 epochs
for i in range(4500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()



In [29]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.4923, grad_fn=<DivBackward0>)


In [30]:
# Predictions
preds

tensor([[ 56.7641,  70.1335],
        [ 82.4764, 100.7487],
        [118.7174, 133.0587],
        [ 21.1204,  37.0578],
        [101.8969, 119.0381]], grad_fn=<AddBackward0>)

In [31]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [32]:
!pip install jovian --upgrade -q

## Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [33]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [34]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [35]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [36]:
from torch.utils.data import TensorDataset

In [37]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [38]:
from torch.utils.data import DataLoader

In [39]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a `for-in` loop. Let's look at an example.

In [40]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.]])
tensor([[119., 133.],
        [ 56.,  70.],
        [119., 133.],
        [ 81., 101.],
        [103., 119.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [41]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4508, -0.0088,  0.5288],
        [-0.0167,  0.1606,  0.0107]], requires_grad=True)
Parameter containing:
tensor([-0.3672, -0.0017], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [42]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4508, -0.0088,  0.5288],
         [-0.0167,  0.1606,  0.0107]], requires_grad=True),
 Parameter containing:
 tensor([-0.3672, -0.0017], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [43]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-11.1234,   9.9966],
        [ -8.3170,  13.2918],
        [-10.0891,  20.6804],
        [-27.1588,   5.5956],
        [  4.7029,  15.0071],
        [-11.1234,   9.9966],
        [ -8.3170,  13.2918],
        [-10.0891,  20.6804],
        [-27.1588,   5.5956],
        [  4.7029,  15.0071],
        [-11.1234,   9.9966],
        [ -8.3170,  13.2918],
        [-10.0891,  20.6804],
        [-27.1588,   5.5956],
        [  4.7029,  15.0071]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [44]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [45]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [46]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7693.5557, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [47]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [48]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update, and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [49]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 116.3680
Epoch [20/100], Loss: 119.6850
Epoch [30/100], Loss: 117.7008
Epoch [40/100], Loss: 48.8977
Epoch [50/100], Loss: 62.8874
Epoch [60/100], Loss: 35.1469
Epoch [70/100], Loss: 28.2144
Epoch [80/100], Loss: 19.8325
Epoch [90/100], Loss: 19.7516
Epoch [100/100], Loss: 21.3906


In [50]:
mino = []

In [51]:
mino= fit(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 17.9236
Epoch [20/1000], Loss: 15.9234
Epoch [30/1000], Loss: 10.3362
Epoch [40/1000], Loss: 13.6794
Epoch [50/1000], Loss: 8.0386
Epoch [60/1000], Loss: 7.6182
Epoch [70/1000], Loss: 10.2052
Epoch [80/1000], Loss: 7.5002
Epoch [90/1000], Loss: 10.2036
Epoch [100/1000], Loss: 3.4896
Epoch [110/1000], Loss: 8.0296
Epoch [120/1000], Loss: 10.2526
Epoch [130/1000], Loss: 5.8856
Epoch [140/1000], Loss: 4.4442
Epoch [150/1000], Loss: 9.4569
Epoch [160/1000], Loss: 4.0771
Epoch [170/1000], Loss: 2.6884
Epoch [180/1000], Loss: 5.7052
Epoch [190/1000], Loss: 5.2445
Epoch [200/1000], Loss: 7.3640
Epoch [210/1000], Loss: 5.6358
Epoch [220/1000], Loss: 3.6300
Epoch [230/1000], Loss: 3.7772
Epoch [240/1000], Loss: 4.2855
Epoch [250/1000], Loss: 2.6379
Epoch [260/1000], Loss: 3.8164
Epoch [270/1000], Loss: 5.4940
Epoch [280/1000], Loss: 3.5340
Epoch [290/1000], Loss: 2.6107
Epoch [300/1000], Loss: 1.9173
Epoch [310/1000], Loss: 2.4206
Epoch [320/1000], Loss: 3.0708
Epoch [330

In [52]:
print(mino)

None


Let's generate predictions using our model and verify that they're close to our targets.

In [53]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.1079,  70.3456],
        [ 82.3097, 100.4924],
        [118.6148, 133.3828],
        [ 21.0686,  37.1437],
        [102.0297, 118.7480],
        [ 57.1079,  70.3456],
        [ 82.3097, 100.4924],
        [118.6148, 133.3828],
        [ 21.0686,  37.1437],
        [102.0297, 118.7480],
        [ 57.1079,  70.3456],
        [ 82.3097, 100.4924],
        [118.6148, 133.3828],
        [ 21.0686,  37.1437],
        [102.0297, 118.7480]], grad_fn=<AddmmBackward>)

In [54]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.

## Commit and update the notebook

As a final step, we can record a new version of the notebook using the `jovian` library.

In [56]:
import jovian

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit(project='linear-regression-pytorch')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Detected Kaggle notebook...
[jovian] Please enter your API key ( from https://jovian.ml/ ):
API KEY: ········
[jovian] Uploading notebook to https://jovian.ml/satnam00/linear-regression-pytorch


<IPython.core.display.Javascript object>